# RFM Segmentation
https://clevertap.com/blog/rfm-analysis/

## 1. Data Acquisition & Analysis

## 2. Modeling - Data Preparation

In [99]:
val startTime = System.nanoTime()

startTime = 50593955851584


50593955851584

In [100]:
import org.apache.spark.sql.types._

val schema = StructType(
                List(
                    StructField("customer_id", StringType, false),
                    StructField("purchase_amount", DoubleType, false),
                    StructField("date_of_purchase", DateType, false)
                )
            )
val data = spark.read
                .option("sep", "\t")
                .option("mode","FAILFAST")
                .option("dateFormat","YYYY-MM-dd")
                .schema(schema)
                .csv("../../data/foundation-marketing-analytics/purchases.txt")
                .toDF

schema = StructType(StructField(customer_id,StringType,false), StructField(purchase_amount,DoubleType,false), StructField(date_of_purchase,DateType,false))
data = [customer_id: string, purchase_amount: double ... 1 more field]


[customer_id: string, purchase_amount: double ... 1 more field]

### Creating

#### Derive days_since col for RECENCY calculation later

In [101]:
import org.apache.spark.sql.functions._

//Why is end_date set to "2016-01-01"?

val enriched1 = data
                .withColumn("end_date", lit("2016-01-01"))
                .withColumn("days_since", datediff($"end_date", $"date_of_purchase"))
enriched1.show(5)

//Verify if any calculations have failed
val nullCount = enriched1.filter(isnull($"days_since")).count()
assert(nullCount == 0)

+-----------+---------------+----------------+----------+----------+
|customer_id|purchase_amount|date_of_purchase|  end_date|days_since|
+-----------+---------------+----------------+----------+----------+
|        760|           25.0|      2009-11-06|2016-01-01|      2247|
|        860|           50.0|      2012-09-28|2016-01-01|      1190|
|       1200|          100.0|      2005-10-25|2016-01-01|      3720|
|       1420|           50.0|      2009-07-09|2016-01-01|      2367|
|       1940|           70.0|      2013-01-25|2016-01-01|      1071|
+-----------+---------------+----------------+----------+----------+
only showing top 5 rows



enriched1 = [customer_id: string, purchase_amount: double ... 3 more fields]
nullCount = 0


0

#### Create features: FREQUENCY, RECENCY (in days) and Monetary value

1. Recency: How recently a customer has made a purchase
2. Frequency: How often a customer makes a purchase
3. Monetary Value: How much money a customer spends on purchases

In [102]:
val enriched2 = enriched1
                .groupBy($"customer_id")
                .agg(
                    min($"days_since").alias("recency"),
                    count($"customer_id").alias("frequency"),
                    avg($"purchase_amount").alias("amount"))
enriched2.filter($"customer_id".isin("10", "90")).show(5)

+-----------+-------+---------+------+
|customer_id|recency|frequency|amount|
+-----------+-------+---------+------+
|         90|    758|       10| 115.8|
|         10|   3829|        1|  30.0|
+-----------+-------+---------+------+



enriched2 = [customer_id: string, recency: int ... 2 more fields]


[customer_id: string, recency: int ... 2 more fields]

**Let us do some summary/descriptive stats**

In [103]:
enriched2.describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|       customer_id|           recency|         frequency|            amount|
+-------+------------------+------------------+------------------+------------------+
|  count|             18417|             18417|             18417|             18417|
|   mean|137573.51088668077|  1253.03789976652|2.7823749796383774|57.792985101815624|
| stddev|  69504.5998805089|1081.4378683668397| 2.936888270392829|154.36010930845674|
|    min|                10|                 1|                 1|               5.0|
|    max|             99990|              4014|                45|            4500.0|
+-------+------------------+------------------+------------------+------------------+



In [104]:
enriched2.createOrReplaceTempView("customers")
spark.sql("select * from customers").show()

+-----------+-------+---------+------------------+
|customer_id|recency|frequency|            amount|
+-----------+-------+---------+------------------+
|       6240|   3005|        3| 76.66666666666667|
|      52800|   3320|        1|              15.0|
|     100140|     13|        4|             51.25|
|     109180|     30|        8|             48.75|
|     131450|    205|        8|            103.75|
|      45300|    234|        6|29.166666666666668|
|      69460|     15|        9| 28.88888888888889|
|      86180|      2|        9| 21.11111111111111|
|     161110|   1528|        1|              30.0|
|      60070|   2074|        3|51.666666666666664|
|      13610|   1307|        8|           3043.75|
|     100010|    413|        7|27.857142857142858|
|     107930|    150|        5|              79.0|
|     132610|     30|        7|28.571428571428573|
|     154770|    427|        1|              45.0|
|      49290|    371|        5|              24.0|
|     229650|    419|        1|

### Feature Scaling

We need to address the following before we can train the model:

1. Data Dispersion: The *amount* is skewed. We need to take **log** to address this skewness.
2. Scale: Different features use different scales (see table). These features need to be standardized so that they contribute equally to the result.


|Feature|Scale|
|-------|-----|
|recency|days|
|frequency|purchase occassions|
|amount|dollars|

**1. Handle data dispersion**

Below histogram shows that majority of the purchases are below 5 and 300 dollars. The data is left-skewed .

In [105]:
val (startValues, counts) = enriched2.select($"amount").map(v=>v.getDouble(0)).rdd.histogram(15)

startValues = Array(5.0, 304.6666666666667, 604.3333333333334, 904.0, 1203.6666666666667, 1503.3333333333333, 1803.0, 2102.6666666666665, 2402.3333333333335, 2702.0, 3001.6666666666665, 3301.3333333333335, 3601.0, 3900.6666666666665, 4200.333333333333, 4500.0)
counts = Array(18126, 144, 28, 56, 16, 8, 19, 4, 3, 5, 1, 0, 0, 5, 2)


Array(18126, 144, 28, 56, 16, 8, 19, 4, 3, 5, 1, 0, 0, 5, 2)

Let us derive a new column, *log_amount*

In [106]:
val enriched3 = enriched2.withColumn("log_amount", log($"amount"))
val (startValues, counts) = enriched3.select($"log_amount").map(v=>v.getDouble(0)).rdd.histogram(15)
enriched3.createOrReplaceTempView("customers")
spark.sql("select * from customers").show(5)

+-----------+-------+---------+-----------------+------------------+
|customer_id|recency|frequency|           amount|        log_amount|
+-----------+-------+---------+-----------------+------------------+
|       6240|   3005|        3|76.66666666666667| 4.339467020255086|
|      52800|   3320|        1|             15.0|  2.70805020110221|
|     100140|     13|        4|            51.25|3.9367156180185177|
|     109180|     30|        8|            48.75| 3.886705197443856|
|     131450|    205|        8|           103.75| 4.641984159110808|
+-----------+-------+---------+-----------------+------------------+
only showing top 5 rows



enriched3 = [customer_id: string, recency: int ... 3 more fields]
startValues = Array(1.6094379124341003, 2.062930896655721, 2.516423880877342, 2.9699168650989627, 3.423409849320583, 3.8769028335422036, 4.330395817763825, 4.783888801985445, 5.237381786207066, 5.690874770428687, 6.1443677546503075, 6.597860738871929, 7.051353723093549, 7.50484670731517, 7.958339691536791, 8.411832675758411)
counts = Array(109, 987, 1567, 6989, 3004, 3291, 1453, 487, 217, 95, 81, 73, 26, 26, 12)


Array(109, 987, 1567, 6989, 3004, 3291, 1453, 487, 217, 95, 81, 73, 26, 26, 12)

**2. Scale**

Distance algorithms like K-means (clustering) are most affected by the range of features because they are using distance between data points to determine their similarity.  So there is a chance that higher weights will be given to features with higher magnitude.

**Standardization** is a technique used to scale our features so that all the features **contribute equally to the result**. Logic: (data - mean)/std-dev. This puts most of the data between -2 & 2.

We will use [StandardScaler](https://spark.apache.org/docs/1.4.1/ml-features.html#standardscaler) from Spark.

In [107]:
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.util.MLUtils

val assembler = new VectorAssembler()
                    .setInputCols(Array("recency", "frequency", "log_amount"))
                    .setOutputCol("vectors")

val vectors = assembler.transform(enriched3)
features.show(3)


val scaler = new StandardScaler()
                .setInputCol("vectors")
                .setOutputCol("features")
                .setWithStd(true)
                .setWithMean(true)
// Compute summary statistics by fitting the StandardScaler
val scalerModel = scaler.fit(vectors)
// Standardize each feature to have unit standard deviation.
val scaledFeatures = scalerModel.transform(vectors)
scaledFeatures.show(3, truncate=false)

+-----------+-------+---------+-----------------+------------------+--------------------+
|customer_id|recency|frequency|           amount|        log_amount|            features|
+-----------+-------+---------+-----------------+------------------+--------------------+
|       6240|   3005|        3|76.66666666666667| 4.339467020255086|[3005.0,3.0,4.339...|
|      52800|   3320|        1|             15.0|  2.70805020110221|[3320.0,1.0,2.708...|
|     100140|     13|        4|            51.25|3.9367156180185177|[13.0,4.0,3.93671...|
+-----------+-------+---------+-----------------+------------------+--------------------+
only showing top 3 rows

+-----------+-------+---------+-----------------+------------------+------------------------------+-----------------------------------------------------------+
|customer_id|recency|frequency|amount           |log_amount        |vectors                       |features                                                   |
+-----------+-------+----

assembler = vecAssembler_f240288b85fc
vectors = [customer_id: string, recency: int ... 4 more fields]
scaler = stdScal_475506728ef0
scalerModel = stdScal_475506728ef0
scaledFeatures = [customer_id: string, recency: int ... 5 more fields]


[customer_id: string, recency: int ... 5 more fields]

## 2. Modeling - Model Building

Extract the feature vector from *scaledFeatures* dataframe so that we can fit the model

In [108]:
import org.apache.spark.ml.clustering.BisectingKMeans
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Vector

//std_features col is of type vector
scaledFeatures.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- recency: integer (nullable = true)
 |-- frequency: long (nullable = false)
 |-- amount: double (nullable = true)
 |-- log_amount: double (nullable = true)
 |-- vectors: vector (nullable = true)
 |-- features: vector (nullable = true)



We will apply [hierarchial clustering](http://spark.apache.org/docs/latest/mllib-clustering.html#bisecting-k-means) using BisectingKMeans

In [109]:
// Clustering the data into 9 clusters by BisectingKMeans.
val bkm = new BisectingKMeans().setK(9)
val model = bkm.fit(scaledFeatures)

bkm = bisecting-kmeans_6c8e1ab54200
model = bisecting-kmeans_6c8e1ab54200


bisecting-kmeans_6c8e1ab54200

In [144]:
// Show the compute cost and the cluster centers
println(s"Within Set Sum of Squared Errors: ${model.computeCost(scaledFeatures)}")

val summary = model.summary

println(s"Number of clusters: ${summary.clusterSizes.length}")
println(s"Number of iterations: ${summary.numIter}")

println("Cluster centers:")
model.clusterCenters.zipWithIndex.foreach { case (center, idx) =>
  println(s"Cluster Center ${idx}: ${center}")
}
println("Frequency (or number of data points) of each cluster:")
summary.clusterSizes.zipWithIndex.foreach { case(size, idx) =>
    println(s"Cluser: ${idx}, # of members: ${size}")
}

Within Set Sum of Squared Errors: 14319.292007940281
Number of clusters: 9
Number of iterations: 20
Cluster centers:
Cluster Center 0: [-0.8453714912512168,-0.1878697149175627,-0.4576033841040799]
Cluster Center 1: [-0.6480140769348111,-0.20432490122092373,0.4307515929770496]
Cluster Center 2: [-0.5669604365509189,-0.20728045846771836,1.8445461933131673]
Cluster Center 3: [-0.8416225909972415,2.1718402417686407,-0.15558894779212123]
Cluster Center 4: [-0.8453839898457399,1.8889976661701162,1.5212988435939618]
Cluster Center 5: [0.045064047848971646,-0.4027017984943741,-0.6439217331189663]
Cluster Center 6: [0.9001547309328005,-0.4303299639742082,-1.3296655550462693]
Cluster Center 7: [1.0211281927777782,-0.4084553533731202,0.5060895525101492]
Cluster Center 8: [1.617914928825506,-0.4970940188711775,-0.4932730234533967]
Frequency (or number of data points) of each cluster:
Cluser: 0, # of members: 3148
Cluser: 1, # of members: 2666
Cluser: 2, # of members: 1607
Cluser: 3, # of members: 

summary = org.apache.spark.ml.clustering.BisectingKMeansSummary@65fa1276


org.apache.spark.ml.clustering.BisectingKMeansSummary@65fa1276

## 3. Modeling - Model Evaluation

Accuracy is a useful metric in supervised learning, such as classification. However, in case of unsupervised learning (like KMeans, BisectingKMeans), there is no accuracy (as there is no labeled data / gold standard you can evaluate against.

However, we can use the ClusteringEvaluator for assessing the quality of our model. You can find more information in the docs: https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/evaluation/ClusteringEvaluator.html

## 4. Segmentation summary

In [145]:
summary.predictions.printSchema()
println("Prediction for first 10 customers:")
summary
    .predictions
    .withColumn("cust_id_numeric", $"customer_id".cast(IntegerType))
    .orderBy($"cust_id_numeric")
    .select("customer_id", "prediction")
    .limit(10)
    .show(10)

println("Profile of each segment:")
val segments = summary
                .predictions
                .groupBy($"prediction".alias("segment"))
                .agg(
                    min($"recency").alias("min_r"),
                    round(avg($"recency"),2).alias("avg_r"),
                    max($"recency").alias("max_r"),
                    
                    min($"frequency").alias("min_f"),
                    round(avg($"frequency"),2).alias("avg_f"),
                    max($"frequency").alias("max_f"),
                    
                    min($"amount").alias("min_a"),
                    round(avg($"amount"),2).alias("avg_a"),
                    max($"amount").alias("max_a"))
                .orderBy($"segment")
segments.show(10, truncate=false)

root
 |-- customer_id: string (nullable = true)
 |-- recency: integer (nullable = true)
 |-- frequency: long (nullable = false)
 |-- amount: double (nullable = true)
 |-- log_amount: double (nullable = true)
 |-- vectors: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: integer (nullable = false)

Show prediction for first 10 customers:
+-----------+----------+
|customer_id|prediction|
+-----------+----------+
|         10|         8|
|         80|         4|
|         90|         4|
|        120|         5|
|        130|         7|
|        160|         8|
|        190|         7|
|        220|         5|
|        230|         8|
|        240|         0|
+-----------+----------+

Show profile of each segment:
+-------+-----+-------+-----+-----+-----+-----+------------------+------+-----------------+
|segment|min_r|avg_r  |max_r|min_f|avg_f|max_f|min_a             |avg_a |max_a            |
+-------+-----+-------+-----+-----+-----+-----+----------------

segments = [segment: int, min_r: int ... 8 more fields]


[segment: int, min_r: int ... 8 more fields]

In [111]:
val duration = System.nanoTime() - startTime
import java.util.concurrent.TimeUnit

println(TimeUnit.NANOSECONDS.toMinutes(duration) + " mins.")
println(TimeUnit.NANOSECONDS.toSeconds(duration) + " secs.")

2 mins.
177 secs.


duration = 177045787900


177045787900